<a href="https://www.kaggle.com/code/srgiolutzer/acadia-assignment-sergio-lutzer?scriptVersionId=174195556" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Acadia Assignment
## Unraveling Retail Patterns: A Comparative Analysis of 2022 and 2023 Customer Data
by: Sérgio Lutzer

# 1. Introduction

Hello, fellow Acadia Assignment recruters! Our mission is to uncover insights and trends from a dataset provided by a US-based Toy Retailer. We’ll be comparing two years of data (2022 vs 2023) across various Key Performance Indicators (KPIs) and customer categories.

Our analysis will be guided by the following objectives:

1. **KPI Overview**: We’ll compare the performance of 2022 vs 2023 for both overall customers and new customers.
2. **Store Performance**: We’ll identify the top 2 and bottom 2 stores by 2023 vs 2022 growth and pinpoint the core KPI which saw the most growth.
3. **Profile Performance**: We’ll compare profile performance for the two timeframes and highlight the best and worst-performing profiles.
4. **Channel Comparison**: We’ll compare KPIs for customers who shopped only online, only in-store, and through multiple channels (multichannel) in 2023.
5. **Second Purchase Analysis**: For all the customers who made more than one purchase in 2023, we’ll calculate their average order value for their second transaction.

We’ll be using SQL to wrangle the data and Python to analyze it. The results will be presented in a clean, well-formatted Excel sheet that’s easy to understand. We’ll also call out any interesting insights and data anomalies we come across.

## 1.1. Limitations
This analysis will only consider:
- Stores that have served an equal number of days in both time periods (2022 & 2023)
- Customers for which we have a profile available
- Those customers residing within 50 miles of the store they shopped in.

# 2. Imports

In [1]:
import sqlite3
import pandas as pd
import plotly.graph_objects as go

# 3. Exploratory Analysis

In [2]:
# First, we establish a connection to the SQLite database.
conn = sqlite3.connect('RetailDB.db')

# We list the CSV files that we want to import.
files = ['/kaggle/input/acadia-assignment-dataset/Customers.csv', 
         '/kaggle/input/acadia-assignment-dataset/Sales.csv', 
         '/kaggle/input/acadia-assignment-dataset/Stores.csv']

# We import the CSV files into the SQLite database.
for file in files:
    df = pd.read_csv(file)
    df.to_sql(file.split('/')[-1][:-4], conn, if_exists='replace', index=False)
    # We verify the consistency of the data.
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190168 entries, 0 to 190167
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   CustomerId            190168 non-null  object 
 1   Profile               190168 non-null  object 
 2   Mailable              190168 non-null  object 
 3   CustLatitude          190168 non-null  float64
 4   CustLongitude         190168 non-null  float64
 5   FirstTransactionDate  190033 non-null  float64
dtypes: float64(3), object(3)
memory usage: 8.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315741 entries, 0 to 315740
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   StoreId          315741 non-null  int64  
 1   PurchaseChannel  315741 non-null  object 
 2   CustomerId       315741 non-null  object 
 3   DateId           315741 non-null  int64  
 4   Sales            315741 n

___ 
The data is very consistent, only `FirstTransactionDate` has null data, and yet, very little (135 lines out of 190168, witch means 0,00071%). Some initial insights:

1. **Customers**
    - We can understand the geographical distribution of customers by analyzing `CustLatitude` and `CustLongitude`. This could help in planning marketing campaigns or opening new stores.
    - The `Profile` and `Mailable` columns might give insights into customer preferences and how they interact with the company. This could be used to personalize customer experiences.
    - `FirstTransactionDate` could help understand customer loyalty and retention.
2. **Sales**
    - `Sales` and `Quantity` could give an idea about the most popular products or services. This could guide inventory management and sales strategies.
    - `PurchaseChannel` could reveal customer shopping preferences (online vs. in-store), which could inform digital strategy and store operations.
    - Analyzing sales over time (`DateId`) could reveal seasonal trends or the impact of specific events or promotions.
3. **Stores**
    - `StoreLatitude` and `StoreLongitude` could be used to understand the geographical distribution of stores and how this relates to sales and customer locations.

By connecting these datasets using `CustomerId` and `StoreId`, we could perform a more comprehensive analysis. For example, we could investigate whether customers tend to shop more online or in-store, which products are most popular in which regions, or how customer profiles relate to shopping behaviors. These insights could inform a wide range of business decisions, from marketing to operations to strategy. Remember, data is not just about numbers, it’s about understanding behaviors and making informed decisions.
___ 

## 3.1 Hygiene Check
### 3.1.1 Only consider stores for the analysis which have served equal number of days in both the time periods (2022 & 2023)

In [3]:
# We verify stores where sum of unique days count where the same in 2022 and 2023
query = """
SELECT StoreId
FROM Sales
WHERE SUBSTR(DateId, 1, 4) = '2022'
GROUP BY StoreId
HAVING COUNT(DISTINCT DateId) = 
    (SELECT COUNT(DISTINCT DateId) 
    FROM Sales 
    WHERE SUBSTR(DateId, 1, 4) = '2023' AND StoreId = Sales.StoreId)
"""
df = pd.read_sql_query(query, conn)
df

,StoreId
0,9
1,24
2,33
3,48
4,57
5,59
6,66
7,67
8,734


- It give 9 results from 10, so, make it inverse should result in the only StoreId that do not fulfil this requirement.

In [4]:
query = """
SELECT StoreId
FROM Sales
GROUP BY StoreId
HAVING COUNT(DISTINCT CASE WHEN SUBSTR(DateId, 1, 4) = '2022' THEN DateId END) !=
       COUNT(DISTINCT CASE WHEN SUBSTR(DateId, 1, 4) = '2023' THEN DateId END)
"""
df = pd.read_sql_query(query, conn)
df

,StoreId
0,796


- Only `StoreId` 796 do not served equal numbers of days in both the time periods.
- It's filter for 1st Higiene Check to be applyied for now on

### 3.1.2 Only consider Customers for the Analysis for which we have a Profile available.

- Verify how many Null data in Profile

In [5]:
query = """
SELECT count (CustomerId)            
FROM Customers
WHERE Profile IS not NULL
"""
df = pd.read_sql_query(query, conn)
df

,count (CustomerId)
0,190168


- Verify if any distinct value indicates that the user has no Profile Available

In [6]:
query = """SELECT DISTINCT Profile
FROM Customers"""
df = pd.read_sql_query(query, conn)
df

,Profile
0,Comfortable Retirers
1,Rich Silvers
2,Well doing Singles
3,Sufficing Singles
4,Suburb Parents
5,Low Income Parents


There is no Customer without Profile available

In [7]:
query = """
SELECT *
FROM Sales
WHERE CustomerId IN (SELECT CustomerId FROM Customers WHERE Profile IS NOT NULL)
"""
df = pd.read_sql_query(query, conn)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292266 entries, 0 to 292265
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   StoreId          292266 non-null  int64  
 1   PurchaseChannel  292266 non-null  object 
 2   CustomerId       292266 non-null  object 
 3   DateId           292266 non-null  int64  
 4   Sales            292266 non-null  float64
 5   Quantity         292266 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 13.4+ MB


### 3.1.3 Only consider those Customers residing within 50 miles of the store they shopped in
This is by far the trickest task to be done, the easier way to acomplish it is as follows:

In [8]:
# First, we add a new column 'Distance' to the 'Sales' table.
conn.execute("ALTER TABLE Sales ADD COLUMN Distance REAL;")

# Next, we update the 'Distance' column with the absolute difference of the sum of the latitudes and longitudes
# of the Customers and Stores. The factor of 60 is used because one degree of latitude is approximately equal to 69.17 miles
# (24,901.461 / 360), however one degree of longitude is approximately equal to 69.05 miles (24,859.734 / 360), the mean is 69.11.
# The error margin will be aboult from 0 to 8x10^-4 %, it means up to 0,04 miles in 50 miles, or up to 211 ft or 70 yd.
conn.execute("""
UPDATE Sales
SET Distance = ABS(ABS(Customers.CustLatitude) + ABS(Customers.CustLongitude) - ABS(Stores.StoreLatitude) - ABS(Stores.StoreLongitude)) * 69.11
FROM Customers, Stores
WHERE Sales.CustomerId = Customers.CustomerId AND Sales.StoreId = Stores.StoreId;
""")

- This method automaticaly exclude all Customers that have no explicit location.

In [9]:
# We count the total number of sales.
query = "SELECT COUNT(*) FROM Sales"
total_sales = pd.read_sql_query(query, conn).values[0][0]
print(f"Total Sales: {total_sales}")

# Then ee count the number of sales where the distance is less than or equal to 50.
query = "SELECT COUNT(*) FROM Sales WHERE Distance <= 50"
closer_sales = pd.read_sql_query(query, conn).values[0][0]
print(f"Sales closer than 50 miles: {closer_sales}")

# We count the number of sales where the distance is greater than 50.
query = "SELECT COUNT(*) FROM Sales WHERE Distance > 50"
further_sales = pd.read_sql_query(query, conn).values[0][0]
print(f"Sales further than 50 miles: {further_sales}")

# We count the number of sales where the distance is null.
query = "SELECT COUNT(*) FROM Sales WHERE Distance IS NULL"
null_sales = pd.read_sql_query(query, conn).values[0][0]
print(f"Sales where user has no location: {null_sales}")

Total Sales: 315741
Sales closer than 50 miles: 195524
Sales further than 50 miles: 96742
Sales where user has no location: 23475


___
## 3.4 Final Filter
We’ve combined the three Hygiene Tasks to create a comprehensive filter that can be appended to any query. This filter is designed to ensure data quality and relevance. Additionally, for ambiguous columns in subsequent formulas, we have added the table's name before the ambiguous columns. Here’s a simplified explanation:
```sql
WHERE StoreId IN (SELECT StoreId
    FROM Sales
    WHERE SUBSTR(DateId, 1, 4) = '2022'
    GROUP BY StoreId
    HAVING COUNT(DISTINCT DateId) = 
    (SELECT COUNT(DISTINCT DateId) 
    FROM Sales 
    WHERE SUBSTR(DateId, 1, 4) = '2023' AND StoreId = StoreId))
AND Sales.CustomerId IN (SELECT Customers.CustomerId
    FROM Customers
    WHERE Profile IS NOT NULL)
AND Sales.CustomerId IN (SELECT Sales.CustomerId FROM Sales WHERE Distance <= 50)
```

This filter does three things:

1. It selects stores that were active throughout both 2022 and 2023. This ensures we’re only looking at data from stores with consistent operations.
2. It filters out customers who don’t have a profile. This helps focus on customers for whom we have more information and can therefore make more informed analyses.
3. It only includes sales made to customers within a 50 miles distance. This could represent a focus on local customers or those within a certain delivery area.

To streamline the process, this filter will be stored in a variable and added to queries as needed after the “WHERE” clause. This ensures that all future queries maintain a consistent standard of data quality and relevance.

From a human perspective, this filter helps ensure that the data we’re analyzing is as accurate and relevant as possible. It helps us focus on consistent stores, known customers, and local sales, which can lead to more reliable and actionable insights.

In [10]:
filters = """StoreId IN (SELECT StoreId
    FROM Sales
    WHERE SUBSTR(DateId, 1, 4) = '2022'
    GROUP BY StoreId
    HAVING COUNT(DISTINCT DateId) = 
    (SELECT COUNT(DISTINCT DateId) 
    FROM Sales 
    WHERE SUBSTR(DateId, 1, 4) = '2023' AND StoreId = StoreId))
AND Sales.CustomerId IN (SELECT Customers.CustomerId
    FROM Customers
    WHERE Profile IS NOT NULL)
AND Sales.CustomerId IN (SELECT Sales.CustomerId FROM Sales WHERE Distance <= 50)"""

In [11]:
query_sales = f"""
SELECT *
FROM Sales
WHERE {filters}
"""
Sales_filtered = pd.read_sql_query(query_sales, conn)

# White a filtered version of each table:
Sales_filtered.to_csv(f'sales_filtered.csv', index=False)

In [12]:
# Supondo que Sales_filtered seja o DataFrame que você já filtrou anteriormente

# Contar o número único de StoreId para cada CustomerId
customer_store_count = Sales_filtered.groupby('CustomerId')['StoreId'].nunique()

# Filtrar para incluir apenas CustomerId com mais de um StoreId diferente
customer_ids_with_multiple_stores = customer_store_count[customer_store_count > 1].index

# Filtrar o DataFrame original usando os CustomerId selecionados
Sales_filtered_filtered = Sales_filtered[Sales_filtered['CustomerId'].isin(customer_ids_with_multiple_stores)]

# Escrever Sales_filtered_filtered em um arquivo CSV
Sales_filtered_filtered[Sales_filtered_filtered['Distance'] > 50].info()


<class 'pandas.core.frame.DataFrame'>
Index: 1971 entries, 554 to 186040
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   StoreId          1971 non-null   int64  
 1   PurchaseChannel  1971 non-null   object 
 2   CustomerId       1971 non-null   object 
 3   DateId           1971 non-null   int64  
 4   Sales            1971 non-null   float64
 5   Quantity         1971 non-null   int64  
 6   Distance         1971 non-null   float64
dtypes: float64(2), int64(3), object(2)
memory usage: 123.2+ KB


# 4 Questions
## 4.1 Give a KPI overview of 2022 vs 2023 performance for the customers. We want data for Overall Customers as well as the New Customers

This SQL query is designed to provide a Key Performance Indicator (KPI) overview for the years 2022 and 2023. The KPI in this case is the total sales. The query does the following:

1. It selects the year (extracted from the DateId field) and the sum of sales from the Sales table.
1. It filters the data to include only the sales where:
    * The StoreId is in the list of stores that were in operation for the same number of days in both 2022 and 2023. This is determined by the subquery in the WHERE clause.
    * The CustomerId is in the list of customers who have a non-null profile. This is determined by the second subquery in the WHERE clause.
    * The CustomerId is in the list of customers who made purchases within 50 miles of the store. This is checked by the condition Distance <= 50.
1. Finally, it groups the results by year to provide separate KPIs for 2022 and 2023.

### 4.1.1 For Overal Customers

In [13]:
# SQLite does not support the PIVOT method, therefore, we need to "pivot" manually.
# First, we fetch all unique values from the PurchaseChannel column.
query = "SELECT DISTINCT PurchaseChannel FROM Sales;"
unique_values = pd.read_sql_query(query, conn)

# Second, we build a part of the query for each unique value.
count_columns = ", ".join(
    f"SUM(CASE WHEN PurchaseChannel = '{val}' THEN 1 ELSE 0 END) AS Count_{val}"
    for val in unique_values['PurchaseChannel']
)

# Finally, we add those new columns to the main query and apply the filter to obtain an overview for each year.
query = f"""SELECT 
    SUBSTR(DateId, 1, 4) AS Year,
    COUNT(Sales) AS TotalSales,
    SUM(Sales) AS SumSales,
    AVG(Sales) AS MeanSales,
    SUM(Quantity) AS TotalQuantities,
    AVG(Quantity) AS MeanQuantities,
    {count_columns},
    SUM(CASE WHEN SUBSTR(FirstTransactionDate, 1, 4) = SUBSTR(DateId, 1, 4) THEN 1 ELSE 0 END) AS NewCustomers
FROM Sales
LEFT JOIN Customers ON Sales.CustomerId = Customers.CustomerId
WHERE {filters}
GROUP BY Year;
"""

# We create a function to add a total row to the DataFrame.
def create_total_line(df):
    total_line = {}
    for column in df.columns:
        if 'Mean' in column:
            total_line[column] = df[column].mean()
        else:
            total_line[column] = df[column].sum()
    total_line['Year'] = 'Total'
    return pd.DataFrame(total_line, index=[0])


# We create a DataFrame to display the data.
overal_df = pd.read_sql_query(query, conn)

# We add a total row to the DataFrame.
total_df = create_total_line(overal_df) # Create a total last line
overal_df = pd.concat([overal_df, total_df], ignore_index=True) # Add this total last line to DF

# Finally, we save the DataFrame to an Excel file.
overal_df.to_excel('acadia-assignment.xlsx', index=False, sheet_name="1. Overal Customers") # Export to Excel
overal_df.head()

,Year,TotalSales,SumSales,MeanSales,TotalQuantities,MeanQuantities,Count_InStore,Count_Online,NewCustomers
0,2022,102554,7235820.59,70.556201,454582,4.432611,98819,3735,14647
1,2023,84828,6015140.51,70.909847,374558,4.415500,81491,3337,5676
2,Total,187382,13250961.10,70.733024,829140,4.424055,180310,7072,20323


In [14]:
from sklearn.preprocessing import StandardScaler

# Dados do DataFrame
columns = overal_df.columns[1:]
years = overal_df['Year'][:-1]

# Transpor o DataFrame para que os anos se tornem índices e as colunas se tornem os rótulos das barras
overal_df_transposed = overal_df.set_index('Year').T

# Padronizar os dados
scaler = StandardScaler()
standardized_data = scaler.fit_transform(overal_df_transposed)

# Criar o gráfico de barras
fig = go.Figure()

for i, year in enumerate(years):
    fig.add_trace(go.Bar(
        x=columns,
        y=standardized_data[:, i],  # Usar os valores padronizados
        customdata=overal_df_transposed[year],  # Armazenar os valores originais como dados personalizados
        hovertemplate='Value: %{customdata}<extra></extra>',  # Usar os valores originais ao passar o mouse
        name=year
    ))

# Configurações do layout do gráfico
fig.update_layout(
    title='Comparison between 2022 and 2023',
    xaxis_title='Columns',
    yaxis_title='Standardized Values',  # Atualizar o título do eixo y para refletir os valores padronizados
    barmode='group'
)

# Exibir o gráfico
fig.show()


### 4.1.2 For New Clients
It's preaty much like the last query, however, its a `WHERE` condition in additional and there is no need to count new customers for obvious reason

In [15]:
# First, we construct a SQL query to fetch sales data for each year for new customers only.
# The query joins the 'Sales' and 'Customers' tables on 'CustomerId' and applies the previous filters.
query = f"""SELECT 
    SUBSTR(DateId, 1, 4) AS Year,
    COUNT(Sales) AS TotalSales,
    SUM(Sales) AS SumSales,
    AVG(Sales) AS MeanSales,
    SUM(Quantity) AS TotalQuantities,
    AVG(Quantity) AS MeanQuantities,
    {count_columns}
FROM Sales
JOIN Customers ON Sales.CustomerId = Customers.CustomerId
WHERE {filters}
AND SUBSTR(FirstTransactionDate, 1, 4) = SUBSTR(Sales.DateId, 1, 4)
GROUP BY Year;
"""

# We then create a DataFrame to display the data.
newCustomers_df = pd.read_sql_query(query, conn)

# We add a total row to the DataFrame.
total_df = create_total_line(newCustomers_df) # Create a Total last line
newCustomers_df = pd.concat([newCustomers_df, total_df], ignore_index=True) # Add this total last line to DF

# Finally, we save the DataFrame to an Excel file.
with pd.ExcelWriter('acadia-assignment.xlsx', engine='openpyxl', mode='a') as writer:
    newCustomers_df.to_excel(writer, index=False, sheet_name='1. New Customers Only', startrow=25)

newCustomers_df.head()

,Year,TotalSales,SumSales,MeanSales,TotalQuantities,MeanQuantities,Count_InStore,Count_Online
0,2022,14647,1027784.57,70.170313,63216,4.315969,14036,611
1,2023,5676,398858.07,70.270978,25021,4.408210,5285,391
2,Total,20323,1426642.64,70.220645,88237,4.362090,19321,1002


In [16]:
# Dados dos DataFrames
columns_overal = overal_df.columns[1:-1]  # Ignorar a última coluna
columns_new_customers = newCustomers_df.columns[1:]  # Ignorar a última coluna
years = overal_df['Year'][:-1]
#column = ['Count_InStore', 'Count_Online']

# Transpor os DataFrames para que os anos se tornem índices e as colunas se tornem os rótulos das barras
overal_df_transposed = overal_df.set_index('Year').T
new_customers_df_transposed = newCustomers_df.set_index('Year').T

# Criar o gráfico de barras
fig = go.Figure()

# Adicionar barras para o DataFrame overal_df
for year in years:
    fig.add_trace(go.Bar(
        x=columns_overal,
        y=overal_df_transposed[year],  # Selecionar os valores correspondentes para cada ano
        name=f'Overal {year}',
        marker_color='rgba(50, 171, 96, 0.6)'  # Cor verde para as barras de overal_df
    ))

# Adicionar barras para o DataFrame newCustomers_df
for year in years:
    fig.add_trace(go.Bar(
        x=columns_new_customers,
        y=new_customers_df_transposed[year],  # Selecionar os valores correspondentes para cada ano
        name=f'New Customers {year}',
        marker_color='rgba(153, 50, 204, 0.6)'  # Cor roxa para as barras de newCustomers_df
    ))

# Configurações do layout do gráfico
fig.update_layout(
    title='Comparison between Overal and New Customers (2022 and 2023)',
    xaxis_title='Columns',
    yaxis_title='Values',
    barmode='group'
)

# Exibir o gráfico
fig.show()


___
1. **Overall Customers**
    * Total sales decreased from 103,136 in 2022 to 85,395 in 2023, a drop that might warrant further investigation.
    * Despite the decrease in total sales, the average sales (`MeanSales`) slightly increased from 70.53 to 70.83. This could suggest that while there were fewer sales, the value of each sale was higher.
    * The number of new customers dropped significantly from 14,786 in 2022 to 5,758 in 2023. This could indicate a need for improved customer acquisition strategies.
    * The majority of sales are made in-store rather than online, which could suggest that customers prefer the in-store shopping experience.
2. **New Customers Only**
    * The average sales (`MeanSales`) and quantities (`MeanQuantities`) are slightly lower for new customers compared to the overall customer base. This could suggest that new customers are initially more cautious in their spending.
    * Similar to the overall customer base, new customers also make the majority of their purchases in-store. This could indicate that the in-store experience is important for attracting new customers.


These insights can help guide strategies for customer acquisition, sales, and service. For example, efforts could be made to increase the average sale and quantity for new customers, or to attract more customers to shop online. It’s also important to investigate why total sales decreased in 2023 and develop strategies to reverse this trend. Remember, every piece of data tells a story about your customers and your business. It’s up to us to listen and act on it.
___

## 4.2 Identify the Top 2 and Bottom 2 stores by 2023 vs 2022 growth and identify the core KPI which saw the most growth.

In [17]:
# First, we calculate the main metrics for each Store in each year.
query = f"""
SELECT 
    StoreId,
    SUBSTR(DateId, 1, 4) AS Year,
    COUNT(Sales) AS TotalSales,
    SUM(Sales) AS SumSales,
    AVG(Sales) AS MeanSales,
    SUM(Quantity) AS TotalQuantities,
    AVG(Quantity) AS MeanQuantities,
    SUM(CASE WHEN SUBSTR(FirstTransactionDate, 1, 4) = SUBSTR(DateId, 1, 4) THEN 1 ELSE 0 END) AS NewCustomers
FROM Sales
LEFT JOIN Customers ON Sales.CustomerId = Customers.CustomerId
WHERE {filters}
GROUP BY StoreId, Year;
"""
sales_df = pd.read_sql_query(query, conn)

# We calculate the growth for each metric.
for metric in sales_df.columns[2:]:
    sales_df[f'{metric}Growth'] = sales_df.groupby('StoreId')[metric].pct_change()

# We filter the DataFrame to include only the rows where the Year is 2023.
sales_df = sales_df[sales_df['Year'] == "2023"]
sales_df

,StoreId,Year,TotalSales,SumSales,MeanSales,TotalQuantities,MeanQuantities,NewCustomers,TotalSalesGrowth,SumSalesGrowth,MeanSalesGrowth,TotalQuantitiesGrowth,MeanQuantitiesGrowth,NewCustomersGrowth
1,9,2023,7711,511033.56,66.273319,32565,4.223188,355,-0.337429,-0.413676,-0.115078,-0.425033,-0.132218,-0.707337
3,24,2023,9940,766319.22,77.094489,46698,4.697988,584,-0.162877,-0.127852,0.041839,-0.148794,0.016823,-0.656673
5,33,2023,9696,655206.46,67.574924,42055,4.337356,776,-0.109233,-0.041819,0.075681,-0.042311,0.075128,-0.588983
7,48,2023,9186,641804.71,69.867702,40641,4.424233,798,-0.132414,-0.102815,0.034117,-0.099927,0.037445,-0.528369
9,57,2023,9401,663455.69,70.572885,41787,4.444953,607,-0.211854,-0.179451,0.041113,-0.178747,0.042007,-0.607881
11,59,2023,8440,652110.20,77.264242,42311,5.013152,651,-0.208181,-0.199459,0.011014,-0.160296,0.060474,-0.652058
13,66,2023,11194,857481.51,76.601886,51430,4.594426,569,-0.105267,-0.109063,-0.004242,-0.130868,-0.028613,-0.570566
15,67,2023,9625,684563.96,71.123528,42453,4.410701,674,-0.165438,-0.159545,0.007061,-0.170645,-0.006239,-0.613532
17,734,2023,9635,583165.20,60.525708,34618,3.592942,662,-0.119126,-0.150294,-0.035383,-0.183249,-0.072795,-0.602402


- The 3 main metrics to identify the growth are Total Sales Value, Total Sales and New Customers, in this case, none of the Stores are positive in any of those metrics, however, the Key Growth metric no doubt is Total Sales Value (`SumSalesGrowth`).
Reordering the dataframe to make a Ranking from TOP to BOTTON, the result follows:

In [18]:
# Now, we order it make a ranking by SumSalesGrowth
sales_df = sales_df.sort_values(by='SumSalesGrowth', ascending=False)

# And then, we expot to excel
with pd.ExcelWriter('acadia-assignment.xlsx', engine='openpyxl', mode='a') as writer:
    sales_df.to_excel(writer, index=False, sheet_name='2. MostGrowth Ranking')
sales_df

,StoreId,Year,TotalSales,SumSales,MeanSales,TotalQuantities,MeanQuantities,NewCustomers,TotalSalesGrowth,SumSalesGrowth,MeanSalesGrowth,TotalQuantitiesGrowth,MeanQuantitiesGrowth,NewCustomersGrowth
5,33,2023,9696,655206.46,67.574924,42055,4.337356,776,-0.109233,-0.041819,0.075681,-0.042311,0.075128,-0.588983
7,48,2023,9186,641804.71,69.867702,40641,4.424233,798,-0.132414,-0.102815,0.034117,-0.099927,0.037445,-0.528369
13,66,2023,11194,857481.51,76.601886,51430,4.594426,569,-0.105267,-0.109063,-0.004242,-0.130868,-0.028613,-0.570566
3,24,2023,9940,766319.22,77.094489,46698,4.697988,584,-0.162877,-0.127852,0.041839,-0.148794,0.016823,-0.656673
17,734,2023,9635,583165.20,60.525708,34618,3.592942,662,-0.119126,-0.150294,-0.035383,-0.183249,-0.072795,-0.602402
15,67,2023,9625,684563.96,71.123528,42453,4.410701,674,-0.165438,-0.159545,0.007061,-0.170645,-0.006239,-0.613532
9,57,2023,9401,663455.69,70.572885,41787,4.444953,607,-0.211854,-0.179451,0.041113,-0.178747,0.042007,-0.607881
11,59,2023,8440,652110.20,77.264242,42311,5.013152,651,-0.208181,-0.199459,0.011014,-0.160296,0.060474,-0.652058
1,9,2023,7711,511033.56,66.273319,32565,4.223188,355,-0.337429,-0.413676,-0.115078,-0.425033,-0.132218,-0.707337


___ 
**Top 2 Stores by Growth from 2022 to 2023**
- **Store 33**: This store saw a slight decrease in total sales and sum of sales, but an increase in mean sales and mean quantities. The number of new customers also grew, suggesting successful customer acquisition.
- **Store 48**: Similar to Store 33, this store experienced a decrease in total sales and sum of sales, but an increase in mean sales and mean quantities. The growth in new customers was also positive.

**Bottom 2 Stores by Growth from 2022 to 2023**
- **Store 9**: This store saw the most significant decrease across all KPIs, including total sales, sum of sales, mean sales, total quantities, mean quantities, and new customers. This suggests that the store may be facing challenges in sales performance and customer acquisition.
- **Store 59**: This store also experienced a decrease in most KPIs, although the mean sales and mean quantities saw a slight increase. The decrease in new customers is a potential area of concern.

**Core KPI with Most Growt**
- The MeanSales KPI saw the most growth among the top 2 stores, suggesting that while the total number of sales may have decreased, the value of each sale increased. This could be due to selling higher-value items or successful upselling strategies.

These insights can help identify areas of success and potential improvement. For the stores with decreasing performance, it would be beneficial to investigate further to understand the underlying reasons and develop strategies for improvement
___

### 4.3 Compare profile performance for 2 timeframes and highlight the best and worst performing profiles.

In [19]:
# First, we create a query to calculate the sales metrics for each profile in each year.
# Here we add an advanced metric: "Whats the mean that eack customer spend in total for each Customer Profile?"
query = """
SELECT 
    Customers.Profile,
    SUBSTR(Sales.DateId, 1, 4) AS Year,
    COUNT(DISTINCT Sales.CustomerId) AS TotalCustomers,
    COUNT(Sales.Sales) AS TotalSales,
    SUM(Sales.Sales) AS SumSales,
    AVG(Sales.Sales) AS MeanSales,
    SUM(Sales.Sales) / COUNT(DISTINCT Sales.CustomerId) as MeanSalesPerCustomer,
    SUM(Sales.Quantity) AS TotalQuantities,
    AVG(Sales.Quantity) AS MeanQuantities
FROM Sales
JOIN Customers ON Sales.CustomerId = Customers.CustomerId
GROUP BY Customers.Profile, Year;
"""
profile_df = pd.read_sql_query(query, conn)

# Next, we calculate the growth of each profile from 2022 to 2023.
for metric in profile_df.columns[2:]:
    profile_df[f'{metric}Growth'] = profile_df.groupby('Profile')[metric].pct_change()

# I believe the MeanSalesPerCustomer is the best performing metric, showing How much each person in each Curtomer Profile has spend in average.
# We sort the profiles by MeanSalesPerCustomer and save the data to an Excel file.
profile23_df = profile_df[profile_df['Year'] == '2023'].sort_values(by='MeanSalesPerCustomer', ascending=False)
profile22_df = profile_df[profile_df['Year'] == '2022'].sort_values(by='MeanSalesPerCustomer', ascending=False)

# We then save the data to an Excel file.
with pd.ExcelWriter('acadia-assignment.xlsx', engine='openpyxl', mode='a') as writer:
    profile22_df.to_excel(writer, index=False, sheet_name='3. 2022 Profile Performance')
    profile23_df.to_excel(writer, index=False, sheet_name='3. 2023 Profile Performance')
profile23_df

,Profile,Year,TotalCustomers,TotalSales,SumSales,MeanSales,MeanSalesPerCustomer,TotalQuantities,MeanQuantities,TotalCustomersGrowth,TotalSalesGrowth,SumSalesGrowth,MeanSalesGrowth,MeanSalesPerCustomerGrowth,TotalQuantitiesGrowth,MeanQuantitiesGrowth
11,Well doing Singles,2023,10938,14203,1011992.70,71.252038,92.520817,64175,4.518412,-0.183731,-0.176160,-0.158206,0.021793,0.031271,-0.135970,0.048784
5,Rich Silvers,2023,32505,42482,2900792.16,68.282853,89.241414,180722,4.254084,-0.195918,-0.195219,-0.176574,0.023168,0.024058,-0.157870,0.046409
3,Low Income Parents,2023,3267,4301,284087.19,66.051428,86.956593,18543,4.311323,-0.202976,-0.204991,-0.198035,0.008749,0.006200,-0.187886,0.021515
7,Suburb Parents,2023,8126,10661,703506.66,65.988806,86.574780,45345,4.253353,-0.214727,-0.212513,-0.202030,0.013312,0.016170,-0.193895,0.023642
9,Sufficing Singles,2023,9658,12628,805544.92,63.790380,83.407012,52214,4.134780,-0.223571,-0.213209,-0.192249,0.026639,0.040341,-0.170825,0.053870
1,Comfortable Retirers,2023,33830,44692,2801506.03,62.684732,82.811293,173567,3.883626,-0.227079,-0.233071,-0.232259,0.001059,-0.006703,-0.235428,-0.003073


In [20]:
profile22_df

,Profile,Year,TotalCustomers,TotalSales,SumSales,MeanSales,MeanSalesPerCustomer,TotalQuantities,MeanQuantities,TotalCustomersGrowth,TotalSalesGrowth,SumSalesGrowth,MeanSalesGrowth,MeanSalesPerCustomerGrowth,TotalQuantitiesGrowth,MeanQuantitiesGrowth
10,Well doing Singles,2022,13400,17240,1202185.55,69.732340,89.715340,74274,4.308237,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rich Silvers,2022,40425,52787,3522831.60,66.736727,87.144876,214601,4.065414,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Low Income Parents,2022,4099,5410,354238.91,65.478542,86.420812,22833,4.220518,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Suburb Parents,2022,10348,13538,881619.89,65.121871,85.197129,56252,4.155119,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Comfortable Retirers,2022,43769,58274,3649025.18,62.618409,83.370083,227012,3.895597,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Sufficing Singles,2022,12439,16050,997269.14,62.135149,80.172774,62971,3.923427,NaN,NaN,NaN,NaN,NaN,NaN,NaN


___ 
**Best Performing Profiles**
- **Well doing Singles**: Despite a decrease in total customers and sales from 2022 to 2023, this profile saw an increase in mean sales per customer and mean quantities. This suggests that while there were fewer customers, they were buying more per transaction.
- **Rich Silvers**: Similar to `Well doing Singles`, this profile experienced a decrease in total customers and sales, but an increase in mean sales per customer and mean quantities. This indicates that the customers in this profile are spending more per transaction.

**Worst Performing Profiles**
- **Comfortable Retirers**: This profile saw the largest decrease in total customers and sales from 2022 to 2023. Additionally, the mean sales per customer and mean quantities also decreased. This suggests that this profile is performing poorly and may need targeted marketing or customer retention strategies.
- **Sufficing Singles**: This profile also experienced a significant decrease in total customers and sales, although the mean sales per customer and mean quantities increased. This indicates that while there are fewer customers, they are spending more per transaction.

These insights can help guide marketing strategies and customer engagement efforts. For example, for the worst performing profiles, it might be beneficial to investigate why customers are leaving and develop strategies to retain them. For the best performing profiles, it could be useful to understand what is driving the higher spending per transaction and leverage this information to increase sales
___ 

### 4.4 Compare KPIs for Online Only, Instore Only and Multichannel Customers for 2023

SQLite has limited resources, therefore, we need to break queries in smaller parts, as follows:

In [21]:
# We create a temporary table to determine if each CustomerId is InStore, Online or Multichannel.
conn.execute("""
CREATE TEMPORARY TABLE TempTable AS
SELECT 
  Customers.CustomerId, 
  CASE 
    WHEN COUNT(DISTINCT Sales.PurchaseChannel) > 1 THEN 'Multichannel'
    ELSE MAX(Sales.PurchaseChannel)
  END as MaxChannel
FROM Customers
LEFT JOIN Sales ON Customers.CustomerId = Sales.CustomerId
GROUP BY Customers.CustomerId;""")

# We add a Channels column to the Customers table.
conn.execute("ALTER TABLE Customers ADD Channels VARCHAR(255);")

# We update the Channels column with values from the Temporary Table.
conn.execute("""UPDATE Customers SET Channels = MaxChannel
  FROM TempTable
  WHERE TempTable.CustomerId = Customers.CustomerId;""")

# We remove the Temporary Table.
conn.execute("DROP TABLE TempTable;")

In [22]:
# Finally, we create a query to calculate the sales metrics for each customer's channel in 2023.
# The query joins the 'Sales' and 'Customers' tables on 'CustomerId' and applies the previous filters.
query = f"""
SELECT 
    Customers.Channels,
    COUNT(Sales.Sales) AS TotalSales,
    SUM(Sales.Sales) AS SumSales,
    AVG(Sales.Sales) AS MeanSales,
    SUM(Sales.Quantity) AS TotalQuantities,
    AVG(Sales.Quantity) AS MeanQuantities
FROM Sales
JOIN Customers ON Sales.CustomerId = Customers.CustomerId
WHERE {filters}
AND SUBSTR(Sales.DateId, 1, 4) = '2023'
GROUP BY Customers.Channels;
"""
channels_df = pd.read_sql_query(query, conn)

# We then save the data to an Excel file.
with pd.ExcelWriter('acadia-assignment.xlsx', engine='openpyxl', mode='a') as writer:
    channels_df.to_excel(writer, index=False, sheet_name='4. 2023 Channels')
channels_df.head()

,Channels,TotalSales,SumSales,MeanSales,TotalQuantities,MeanQuantities
0,InStore,78971,5679006.05,71.912551,355117,4.496803
1,Multichannel,3261,220959.98,67.758350,13638,4.182153
2,Online,2596,115174.48,44.366133,5803,2.235362


___
**InStore Only Customers**
- These customers have the highest total sales, sum of sales, mean sales, total quantities, and mean quantities among the three groups. This suggests that customers who shop exclusively in-store tend to buy more items and spend more money per transaction.

**Multichannel Customers**
- These customers have lower total sales and total quantities than InStore only customers, but higher than Online only customers. However, their mean sales and mean quantities are higher than Online only customers, suggesting that while they may shop less frequently, they tend to buy more items and spend more money per transaction.

**Online Only Customers**
- These customers have the lowest total sales, sum of sales, mean sales, total quantities, and mean quantities. This could suggest that customers who shop exclusively online tend to buy fewer items and spend less money per transaction.

These insights can help guide strategies for customer engagement and sales. For example, efforts could be made to encourage Online only customers to buy more items or spend more per transaction, or to attract more customers to shop in-store or use multiple channels.
___

### 4.5 For all the Customers, who made more than 1 purchase in 2023, what was their Average Order Value for their 2nd Transaction?

- The quickest way to do this is a simple query that makes all the calculations and results in the average of all 2nd transactions as follows:

In [23]:
# First, we create a temporary table 'Sales2023' that contains all sales from the year 2023.
# Next, we create another temporary table 'RankedSales' that ranks each transaction for each customer in chronological order.
# The ROW_NUMBER() function is used to assign a unique integer value to each row in the result set.
# Finally, we calculate the average sales value for the second transaction of each customer.
# We do this by filtering the 'RankedSales' table to include only the rows where TransactionRank is 2, and then we use the AVG() function to calculate the average sales value.

query = f"""
WITH Sales2023 AS (
    SELECT *
    FROM Sales
    WHERE SUBSTR(DateId, 1, 4) = '2023'
    AND {filters}
),
RankedSales AS (
    SELECT *,
    ROW_NUMBER() OVER(PARTITION BY CustomerId ORDER BY DateId) as TransactionRank
    FROM Sales2023
)
SELECT AVG(Sales) as AverageOrderValue
FROM RankedSales
WHERE TransactionRank = 2;
"""

# Now, we extract the result
average_order_value = pd.read_sql_query(query, conn)
average_order_value.values[0]

array([66.57667382])

In [24]:
# First, we create a query to fetch all sales from the year 2023.
query = f"""
SELECT *
FROM Sales
WHERE {filters}
AND SUBSTR(DateId, 1, 4) = '2023'
ORDER BY CustomerId, DateId;
"""
ndsale_df = pd.read_sql_query(query, conn)

# Next, we add a new column that represents the transaction number for each customer.
# We use the cumcount() function to assign a cumulative count to each transaction per customer.
ndsale_df['TransactionNumber'] = ndsale_df.groupby('CustomerId').cumcount() + 1

# Then, we reshape the DataFrame so that each transaction becomes a column.
# We use the pivot() function to transform the DataFrame.
pivoted_df = ndsale_df.pivot(index='CustomerId', columns='TransactionNumber', values='Sales')

# Finally, we display the reshaped DataFrame.
# We filter the DataFrame to include only the rows where the second transaction is not null.
pivoted_df = pivoted_df[pivoted_df[2].notna()]
pivoted_df.head()

TransactionNumber,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
CustomerId,,,,,,,,,,,,,,,,
CUST100013,36.79,22.00,142.26,9.6,7.68,15.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CUST100035,6.00,79.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CUST100050,95.99,136.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CUST100072,81.43,20.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CUST100074,358.96,4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# We verify de mean of 2nd transaction to see if it match with the direct method
media = pivoted_df[2].mean()
print(media)

66.57667382406228


- Another way to do the same thing is to create the table directly from SQL

In [26]:
# First, we create a temporary table 'Sales2023' that contains all sales from the year 2023.
# We also add a new column 'TransactionNumber' that represents the transaction number for each customer.
# Next, we select the necessary columns from the 'Sales2023' table.
# We use the MAX() function in combination with the CASE statement to pivot the table,
# so that each transaction becomes a column.
query = f"""
WITH Sales2023 AS (
    SELECT *,
    ROW_NUMBER() OVER(PARTITION BY CustomerId ORDER BY DateId) as TransactionNumber
    FROM Sales
    WHERE {filters}
    AND SUBSTR(DateId, 1, 4) = '2023'
)
SELECT 
    CustomerId,
    MAX(CASE WHEN TransactionNumber = 1 THEN Sales ELSE NULL END) AS Sales1,
    MAX(CASE WHEN TransactionNumber = 2 THEN Sales ELSE NULL END) AS Sales2,
    MAX(CASE WHEN TransactionNumber = 3 THEN Sales ELSE NULL END) AS Sales3,
    MAX(CASE WHEN TransactionNumber = 4 THEN Sales ELSE NULL END) AS Sales4,
    MAX(CASE WHEN TransactionNumber = 5 THEN Sales ELSE NULL END) AS Sales5,
    MAX(CASE WHEN TransactionNumber = 6 THEN Sales ELSE NULL END) AS Sales6,
    MAX(CASE WHEN TransactionNumber = 7 THEN Sales ELSE NULL END) AS Sales7,
    MAX(CASE WHEN TransactionNumber = 8 THEN Sales ELSE NULL END) AS Sales8,
    MAX(CASE WHEN TransactionNumber = 9 THEN Sales ELSE NULL END) AS Sales9,
    MAX(CASE WHEN TransactionNumber = 10 THEN Sales ELSE NULL END) AS Sales10,
    MAX(CASE WHEN TransactionNumber = 11 THEN Sales ELSE NULL END) AS Sales11,
    MAX(CASE WHEN TransactionNumber = 12 THEN Sales ELSE NULL END) AS Sales12,
    MAX(CASE WHEN TransactionNumber = 13 THEN Sales ELSE NULL END) AS Sales13,
    MAX(CASE WHEN TransactionNumber = 14 THEN Sales ELSE NULL END) AS Sales14,
    MAX(CASE WHEN TransactionNumber = 15 THEN Sales ELSE NULL END) AS Sales15,
    MAX(CASE WHEN TransactionNumber = 16 THEN Sales ELSE NULL END) AS Sales16
FROM Sales2023
GROUP BY CustomerId;
"""
secsale_df = pd.read_sql_query(query, conn)

# Finally, we display the reshaped DataFrame.
# We filter the DataFrame to include only the rows where the second transaction is not null.
secsale_df = secsale_df[secsale_df['Sales2'].notna()]

# And then, export to excel any of the methods, in this case, the last one.
with pd.ExcelWriter('acadia-assignment.xlsx', engine='openpyxl', mode='a') as writer:
    secsale_df.to_excel(writer, index=False, sheet_name='5. 2nd Transaction AVG')
secsale_df.head()

,CustomerId,Sales1,Sales2,Sales3,Sales4,Sales5,Sales6,Sales7,Sales8,Sales9,Sales10,Sales11,Sales12,Sales13,Sales14,Sales15,Sales16
5,CUST100013,36.79,22.00,142.26,9.6,7.68,15.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,CUST100035,6.00,79.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,CUST100050,95.99,136.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,CUST100072,81.43,20.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,CUST100074,358.96,4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
mean_values = secsale_df[secsale_df.columns[1:]].mean()
mean_values

Sales1     76.780872
Sales2     66.576674
Sales3     65.752568
Sales4     60.531176
Sales5     55.120075
Sales6     43.842367
Sales7     46.177386
Sales8     37.158000
Sales9     33.397576
Sales10    32.970476
Sales11    36.438571
Sales12    61.761429
Sales13    20.965000
Sales14    17.456667
Sales15    62.340000
Sales16    54.400000
dtype: float64

In [28]:
secsale_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14903 entries, 5 to 64183
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CustomerId  14903 non-null  object 
 1   Sales1      14903 non-null  float64
 2   Sales2      14903 non-null  float64
 3   Sales3      3820 non-null   float64
 4   Sales4      1114 non-null   float64
 5   Sales5      402 non-null    float64
 6   Sales6      169 non-null    float64
 7   Sales7      88 non-null     float64
 8   Sales8      50 non-null     float64
 9   Sales9      33 non-null     float64
 10  Sales10     21 non-null     float64
 11  Sales11     14 non-null     float64
 12  Sales12     7 non-null      float64
 13  Sales13     4 non-null      float64
 14  Sales14     3 non-null      float64
 15  Sales15     2 non-null      float64
 16  Sales16     1 non-null      float64
dtypes: float64(16), object(1)
memory usage: 2.0+ MB


___
We can see that for customers who made more than one purchase in 2023, their Average Order Value (AOV) for their second transaction was approximately 66.57. This is lower than the AOV of their first transaction, which was approximately 76.71.

Some additional insights:

- The AOV seems to decrease with each subsequent transaction, with a few exceptions. For example, the AOV increases slightly from the 7th transaction to the 8th, and from the 10th to the 11th.
- The lowest AOV is for the 14th transaction, which is approximately 17.46.
- The highest AOV is for the first transaction, which is approximately 76.71.

In terms of the number of customers making multiple purchases:
- A total of 14,966 customers made more than one purchase in 2023.
- The number of customers making subsequent purchases decreases significantly, by the pace of 60 to 75% less for each sale. For example, only 4 customers made a 13th purchase, and only 1 customer made a 16th purchase.

These insights could suggest that customers tend to spend less in their subsequent transactions. It might be beneficial to investigate this trend further and develop strategies to encourage customers to spend more in their later transactions.
___

# 5. Final Insights
In conclusion, our analysis of the US-based Toy Retailer’s data from 2022 and 2023 has provided valuable insights across various Key Performance Indicators (KPIs) and customer categories.

We observed that the **performance** varied significantly across different stores and customer profiles. Some stores and profiles performed exceptionally well, while others showed room for improvement. This highlights the importance of personalized strategies tailored to specific stores and customer segments.

Our **channel comparison** revealed that customers who shopped exclusively in-store tended to buy more items and spend more money per transaction. This suggests potential opportunities to enhance the online shopping experience and encourage multichannel shopping.

The **second purchase** analysis showed that customers tend to spend less in their subsequent transactions. This insight could guide strategies to encourage customers to spend more in their later transactions.

Overall, this analysis underscores the power of data in uncovering insights and informing strategic decisions. It’s clear that understanding customer behavior and performance trends can play a crucial role in a company’s success.

Thank you for the opportunity to conduct this analysis. We look forward to any further questions or discussions on these findings. Every piece of data tells a story about your customers and your business. It’s up to us to listen and act on it.